In [ ]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, random_split
from torchvision import transforms
from PIL import Image, ImageFile
import numpy as np
ImageFile.LOAD_TRUNCATED_IMAGES = True

# Utility function to find common embryo IDs
def get_common_embryo_ids(base_paths):
    sets_of_ids = []
    for path in base_paths:
        subfolders = [d for d in os.listdir(path) if os.path.isdir(os.path.join(path, d))]
        sets_of_ids.append(set(subfolders))
    common_ids = set.intersection(*sets_of_ids)
    return sorted(list(common_ids))

# U-Net building blocks
class DoubleConv(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(DoubleConv, self).__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True),
            nn.Conv2d(out_channels, out_channels, kernel_size=3, padding=1),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True)
        )
    
    def forward(self, x):
        return self.conv(x)

class UNet(nn.Module):
    def __init__(self, in_channels=6, out_channels=1):
        super(UNet, self).__init__()
        self.conv1 = DoubleConv(in_channels, 64)
        self.pool1 = nn.MaxPool2d(2)
        self.conv2 = DoubleConv(64, 128)
        self.pool2 = nn.MaxPool2d(2)
        self.conv3 = DoubleConv(128, 256)
        self.pool3 = nn.MaxPool2d(2)
        self.conv4 = DoubleConv(256, 512)
        self.pool4 = nn.MaxPool2d(2)
        self.conv5 = DoubleConv(512, 1024)
        self.up6 = nn.ConvTranspose2d(1024, 512, kernel_size=2, stride=2)
        self.conv6 = DoubleConv(1024, 512)
        self.up7 = nn.ConvTranspose2d(512, 256, kernel_size=2, stride=2)
        self.conv7 = DoubleConv(512, 256)
        self.up8 = nn.ConvTranspose2d(256, 128, kernel_size=2, stride=2)
        self.conv8 = DoubleConv(256, 128)
        self.up9 = nn.ConvTranspose2d(128, 64, kernel_size=2, stride=2)
        self.conv9 = DoubleConv(128, 64)
        self.conv10 = nn.Conv2d(64, out_channels, kernel_size=1)
    
    def forward(self, x):
        c1 = self.conv1(x)
        p1 = self.pool1(c1)
        c2 = self.conv2(p1)
        p2 = self.pool2(c2)
        c3 = self.conv3(p2)
        p3 = self.pool3(c3)
        c4 = self.conv4(p3)
        p4 = self.pool4(c4)
        c5 = self.conv5(p4)
        up_6 = self.up6(c5)
        merge6 = torch.cat([up_6, c4], dim=1)
        c6 = self.conv6(merge6)
        up_7 = self.up7(c6)
        merge7 = torch.cat([up_7, c3], dim=1)
        c7 = self.conv7(merge7)
        up_8 = self.up8(c7)
        merge8 = torch.cat([up_8, c2], dim=1)
        c8 = self.conv8(merge8)
        up_9 = self.up9(c8)
        merge9 = torch.cat([up_9, c1], dim=1)
        c9 = self.conv9(merge9)
        output = self.conv10(c9)
        return torch.sigmoid(output)

# Dataset with updated target
class EmbryoFocusStackDataset(Dataset):
    def __init__(self, base_paths, embryo_ids, transform=None):
        if len(base_paths) != 6:
            raise ValueError("We need exactly 6 focal-plane directories.")
        self.base_paths = base_paths
        self.embryo_ids = embryo_ids
        self.transform = transform

    def __len__(self):
        return len(self.embryo_ids)

    def __getitem__(self, idx):
        embryo_id = self.embryo_ids[idx]
        focal_images = []
        for path in self.base_paths:
            embryo_subfolder = os.path.join(path, embryo_id)
            image_files = sorted(
                [f for f in os.listdir(embryo_subfolder)
                 if f.lower().endswith(('.jpg', '.jpeg', '.png'))]
            )
            if not image_files:
                raise FileNotFoundError(f"No image found in {embryo_subfolder}")
            img_path = os.path.join(embryo_subfolder, image_files[0])
            image = Image.open(img_path).convert('L')
            if self.transform:
                image = self.transform(image)
            focal_images.append(image)
        
        input_tensor = torch.cat(focal_images, dim=0)
        # Use average of focal images as target
        target = torch.mean(torch.stack(focal_images), dim=0)
        return input_tensor, target

# Loss functions
class DiceLoss(nn.Module):
    def __init__(self, smooth=1):
        super(DiceLoss, self).__init__()
        self.smooth = smooth

    def forward(self, inputs, targets):
        inputs = inputs.view(-1)
        targets = targets.view(-1)
        intersection = (inputs * targets).sum()
        dice_score = (2. * intersection + self.smooth) / (inputs.sum() + targets.sum() + self.smooth)
        return 1 - dice_score

dice_loss_fn = DiceLoss()
l1_loss_fn = nn.L1Loss()

def combined_loss(output, target):
    dice = dice_loss_fn(output, target)
    l1 = l1_loss_fn(output, target)
    total_loss = dice + l1
    return total_loss

# Updated training function
def train_model(model, train_loader, val_loader, num_epochs=50, device='cpu'):
    optimizer = optim.Adam(model.parameters(), lr=0.01)  # Increased learning rate
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=5)
    best_val_loss = float('inf')

    for epoch in range(num_epochs):
        model.train()
        running_train_loss = 0.0
        running_dice = 0.0
        running_l1 = 0.0

        for inputs, targets in train_loader:
            inputs, targets = inputs.to(device), targets.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)

            dice_loss = dice_loss_fn(outputs, targets)
            l1_loss = l1_loss_fn(outputs, targets)
            loss = dice_loss + l1_loss  # Use Dice + L1 Loss
            loss.backward()
            optimizer.step()

            running_train_loss += loss.item() * inputs.size(0)
            running_dice += dice_loss.item() * inputs.size(0)
            running_l1 += l1_loss.item() * inputs.size(0)

        train_loss = running_train_loss / len(train_loader.dataset)
        train_dice = running_dice / len(train_loader.dataset)
        train_l1 = running_l1 / len(train_loader.dataset)

        # Validation
        model.eval()
        running_val_loss = 0.0
        with torch.no_grad():
            for inputs, targets in val_loader:
                inputs, targets = inputs.to(device), targets.to(device)
                outputs = model(inputs)
                loss = combined_loss(outputs, targets)
                running_val_loss += loss.item() * inputs.size(0)

        val_loss = running_val_loss / len(val_loader.dataset)

        print(f"Epoch [{epoch+1}/{num_epochs}] "
              f"Train Loss: {train_loss:.4f} (Dice: {train_dice:.4f}, L1: {train_l1:.4f}), "
              f"Val Loss: {val_loss:.4f}")
        
        scheduler.step(val_loss)  # Adjust learning rate based on validation loss

        if val_loss < best_val_loss:
            best_val_loss = val_loss
            torch.save(model.state_dict(), 'embryo_unet.pth')
            print(f"  [*] Model saved at epoch {epoch+1}")

# Main script
def main():
    base_paths = [
        r"C:\Projects\Embryo\Dataset\embryo_dataset_F15",
        r"C:\Projects\Embryo\Dataset\embryo_dataset_F-15",
        r"C:\Projects\Embryo\Dataset\embryo_dataset_F30",
        r"C:\Projects\Embryo\Dataset\embryo_dataset_F-30",
        r"C:\Projects\Embryo\Dataset\embryo_dataset_F45",
        r"C:\Projects\Embryo\Dataset\embryo_dataset_F-45"
    ]
    
    embryo_ids = get_common_embryo_ids(base_paths)
    print(f"Found {len(embryo_ids)} embryo IDs: {embryo_ids[:5]} ...")
    
    transform = transforms.Compose([
        transforms.Resize((256, 256)),
        transforms.ToTensor()
    ])
    
    dataset = EmbryoFocusStackDataset(base_paths, embryo_ids, transform=transform)
    train_size = int(0.8 * len(dataset))
    val_size = len(dataset) - train_size
    train_dataset, val_dataset = random_split(dataset, [train_size, val_size])
    
    train_loader = DataLoader(train_dataset, batch_size=2, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=2, shuffle=False)
    
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"Using device: {device}")
    
    model = UNet(in_channels=6, out_channels=1).to(device)
    
    print("Starting training...")
    train_model(model, train_loader, val_loader, num_epochs=50, device=device)
    print("Training complete. Best model saved as 'embryo_unet.pth'.")

if __name__ == "__main__":
    main()

Found 704 embryo IDs: ['AA83-7', 'AAL839-6', 'AB028-6', 'AB91-1', 'AC264-1'] ...
Using device: cuda
Starting training...
Epoch [1/50] Train Loss: 0.6469 (Dice: 0.5513, L1: 0.0956), Val Loss: 0.6181
  [*] Model saved at epoch 1
Epoch [2/50] Train Loss: 0.6404 (Dice: 0.5496, L1: 0.0907), Val Loss: 0.6202
Epoch [3/50] Train Loss: 0.6334 (Dice: 0.5475, L1: 0.0859), Val Loss: 0.6191
Epoch [4/50] Train Loss: 0.6296 (Dice: 0.5460, L1: 0.0836), Val Loss: 0.6145
  [*] Model saved at epoch 4
Epoch [5/50] Train Loss: 0.6379 (Dice: 0.5511, L1: 0.0868), Val Loss: 0.6332
Epoch [6/50] Train Loss: 0.6364 (Dice: 0.5499, L1: 0.0865), Val Loss: 0.6154
Epoch [7/50] Train Loss: 0.6367 (Dice: 0.5486, L1: 0.0880), Val Loss: 0.6210
Epoch [8/50] Train Loss: 0.6339 (Dice: 0.5491, L1: 0.0848), Val Loss: 0.6341
Epoch [9/50] Train Loss: 0.6364 (Dice: 0.5502, L1: 0.0862), Val Loss: 0.6175
Epoch [10/50] Train Loss: 0.6335 (Dice: 0.5481, L1: 0.0854), Val Loss: 0.6224
Epoch [11/50] Train Loss: 0.6280 (Dice: 0.5514, L1

In [1]:
def test_single_embryo(model, image_paths, transform, device='cpu'):
    """
    Loads 6 images from different focal planes, stacks them as 6-channel input,
    and returns the fused output image (as a PIL Image).
    
    Args:
        model (nn.Module): Trained UNet model with 6 input channels.
        image_paths (list[str]): List of 6 image file paths (one per focal plane).
        transform (callable): Same transform used in training (resize, ToTensor, etc.).
        device (str): 'cpu' or 'cuda'.
        
    Returns:
        PIL.Image: The fused output image (single-channel).
    """
    # Make sure the model is in eval mode
    model.eval()
    
    # Load and transform each focal-plane image
    focal_tensors = []
    for path in image_paths:
        # Open and convert to grayscale
        img = Image.open(path).convert('L')
        # Apply the same transform (e.g., resize, ToTensor)
        img_tensor = transform(img)
        focal_tensors.append(img_tensor)
    
    # Stack along channel dimension: shape (6, H, W)
    input_tensor = torch.cat(focal_tensors, dim=0)
    # Add batch dimension: shape (1, 6, H, W)
    input_tensor = input_tensor.unsqueeze(0).to(device)
    
    # Forward pass through the model
    with torch.no_grad():
        output = model(input_tensor)  # shape (1, 1, H, W)
    
    # Convert the output tensor back to a PIL image
    output_image = output.squeeze(0).cpu()  # shape (1, H, W)
    fused_pil = transforms.ToPILImage()(output_image)
    
    return fused_pil

def main_test():
    """
    Sets up paths, loads the trained model, generates the fused image for a test embryo,
    and optionally compares it with the corresponding 'best' image.
    """
    # 1) Paths to your 6 focal-plane images for one embryo
    test_image_paths = [
        r"C:\Projects\Embryo\Dataset\embryo_dataset_F15\AB91-1\D2013.01.29_S0719_I132_WELL1_RUN169.jpeg",
        r"C:\Projects\Embryo\Dataset\embryo_dataset_F-15\AB91-1\D2013.01.29_S0719_I132_WELL1_RUN169.jpeg",
        r"C:\Projects\Embryo\Dataset\embryo_dataset_F45\AB91-1\D2013.01.29_S0719_I132_WELL1_RUN169.jpeg",
        r"C:\Projects\Embryo\Dataset\embryo_dataset_F-45\AB91-1\D2013.01.29_S0719_I132_WELL1_RUN169.jpeg",
        r"C:\Projects\Embryo\Dataset\embryo_dataset_F30\AB91-1\D2013.01.29_S0719_I132_WELL1_RUN169.jpeg",
        r"C:\Projects\Embryo\Dataset\embryo_dataset_F-30\AB91-1\D2013.01.29_S0719_I132_WELL1_RUN169.jpeg"
    ]
    
    # 2) Define the same transform used in training
    transform = transforms.Compose([
        transforms.Resize((256, 256)),
        transforms.ToTensor()
    ])
    
    # 3) Load the trained model
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = UNet(in_channels=6, out_channels=1).to(device)
    
    if os.path.exists('embryo_unet.pth'):
        model.load_state_dict(torch.load('embryo_unet.pth', map_location=device))
        print("Loaded trained model weights.")
    else:
        print("Warning: No trained model found. Using random weights.")
    
    # 4) Generate the fused image
    fused_image = test_single_embryo(model, test_image_paths, transform, device)
    
    # 5) Load the corresponding "best" image for comparison (adjust path as needed)
    best_image_path = r"C:\Projects\Embryo\Dataset\best_images\AB91-1_best.jpg"
    if os.path.exists(best_image_path):
        best_image = Image.open(best_image_path).convert('L')
        best_image = transform(best_image)
        best_pil = transforms.ToPILImage()(best_image)
        
        # Save both images
        fused_image.save("fused_output.jpg")
        best_pil.save("best_image.jpg")
        
        # Display both images for comparison
        fig, axes = plt.subplots(1, 2, figsize=(10, 5))
        axes[0].imshow(fused_image, cmap='gray')
        axes[0].set_title("Model Output")
        axes[0].axis('off')
        axes[1].imshow(best_pil, cmap='gray')
        axes[1].set_title("Best Image")
        axes[1].axis('off')
        plt.show()
    else:
        # If no "best" image is available, just save and show the fused image
        fused_image.save("fused_output.jpg")
        fused_image.show()

if __name__ == "__main__":
    main_test()

NameError: name 'transforms' is not defined